In [2]:
# Sentiment Analysis

In [3]:
import pandas as pd

for big files when commiting to github https://stackoverflow.com/questions/32953238/how-can-i-ignore-big-files-and-push-to-git-repo


In [10]:

col_names = ['sentiment', 'id', 'date', 'query_string', 'user', 'text']
data_path = '/home/cate/streamlit-projects/sentiment140/training.1600000.processed.noemoticon.csv'

In [11]:
tweet_data = pd.read_csv(data_path, header = None, names = col_names, encoding="ISO-8859-1",engine = 'python' ).sample(frac = 1) #shaffles the data
tweet_data = tweet_data[['sentiment', 'text']] #disregard other columns
print(tweet_data.head())

sentiment                                               text
191614           0  Yeah I Hope So!! Its To Hard To See Him N Just...
1149676          4  And a perfect rainbow never seemed so dull. An...
128223           0                 My tummy hurts and I'm dehydrated 
767517           0  No sooner than I leave the office with the hop...
180436           0  A Friday night down the drain! Damn you 8 am m...


# Data Preprocessing

In [12]:
import re

allowed_chars = ' AaBbCcDdEeFfGgHhIiJjKkLlMmNnOoPpQqRrSsTtUuVvWwXxYyZz0123456789~`!@#$%^&*()-=_+[]{}|;:",./<>?'
punct = '!?,.@#'
maxlen = 280

def preprocess(text):
    return ''.join([' ' + char + ' ' if char in punct else char for char in [char for char in re.sub(r'http\S+', 'http', text, flags=re.MULTILINE) if char in allowed_chars]])[:maxlen]


Flair requires a specific format for the data to be in; it wants three CSV’s that look like:

\__label__ < LABEL>      < TEXT>

In [13]:
#apply the preprocessing function
tweet_data['text'] = tweet_data['text'].apply(preprocess)

In [14]:
# put __label__ in front of each sentiment
tweet_data['sentiment'] = '__label__' + tweet_data['sentiment'].astype(str)

In [15]:
#save the data
import os

# Create directory for saving data if it does not already exist
data_dir = '/home/cate/streamlit-projects/twitter_sentiment/processed-data/'
if not os.path.isdir(data_dir):
    os.mkdir(data_dir)

# Save a percentage of the data (you could also only load a fraction of the data instead)
amount = 0.125

tweet_data.iloc[0:int(len(tweet_data)*0.8*amount)].to_csv(data_dir + '/train.csv', sep='\t', index=False, header=False)
tweet_data.iloc[int(len(tweet_data)*0.8*amount):int(len(tweet_data)*0.9*amount)].to_csv(data_dir + '/test.csv', sep='\t', index=False, header=False)
tweet_data.iloc[int(len(tweet_data)*0.9*amount):int(len(tweet_data)*1.0*amount)].to_csv(data_dir + '/dev.csv', sep='\t', index=False, header=False)

# Classification

In this particular example we will use **Flair** This is a popular machine learning library for state-of-the-art NLP


Flair tutorials -> https://github.com/flairNLP/flair/tree/master/resources/docs

## Loading the data 
Loading the test, train and dev sets using Flair's NLPTaskDataFetcher class

In [16]:
from flair.datasets import ClassificationCorpus
from pathlib import Path

# load corpus containing training, test and dev data
corpus = ClassificationCorpus(Path(data_dir),
                                      test_file='test.csv',
                                      dev_file='dev.csv',
                                      train_file='train.csv')

2020-05-11 15:25:34,306 Reading data from /home/cate/streamlit-projects/twitter_sentiment/processed-data
2020-05-11 15:25:34,307 Train: /home/cate/streamlit-projects/twitter_sentiment/processed-data/train.csv
2020-05-11 15:25:34,307 Dev: /home/cate/streamlit-projects/twitter_sentiment/processed-data/dev.csv
2020-05-11 15:25:34,308 Test: /home/cate/streamlit-projects/twitter_sentiment/processed-data/test.csv


In [17]:
# a label dictionary to hold all the labels assigned to the text in the corpus

label_dict = corpus.make_label_dictionary()

2020-05-11 15:25:36,721 Computing label dictionary. Progress:
100%|██████████| 160000/160000 [00:33<00:00, 4836.88it/s]2020-05-11 15:26:09,947 [b'0', b'4']



In [19]:
# loading Flair's provided GloVe embeddings

from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentRNNEmbeddings

word_embeddings = [WordEmbeddings('glove'),
# these are extra word embeddings provided by Flair for superior results
#                    FlairEmbeddings('news-forward'),
#                    FlairEmbeddings('news-backward')
                  ]

In [20]:
#initialize the embeddings

document_embeddings = DocumentRNNEmbeddings(word_embeddings, hidden_size=512, reproject_words=True, reproject_words_dimension=256)

In [21]:
from flair.models import TextClassifier

classifier = TextClassifier(document_embeddings, label_dictionary=label_dict)

In [22]:
from flair.trainers import ModelTrainer

trainer = ModelTrainer(classifier, corpus)

In [23]:
#train the model
trainer.train('model-saves',
              learning_rate=0.1,
              mini_batch_size=32,
              anneal_factor=0.5,
              patience=8,
              max_epochs=15)

00/5000 - loss 0.64147999 - samples/sec: 130.88
2020-05-11 15:42:33,072 epoch 1 - iter 3500/5000 - loss 0.63742109 - samples/sec: 128.95
2020-05-11 15:44:33,132 epoch 1 - iter 4000/5000 - loss 0.63288279 - samples/sec: 139.41
2020-05-11 15:46:44,634 epoch 1 - iter 4500/5000 - loss 0.62962353 - samples/sec: 126.95
2020-05-11 15:48:38,764 ----------------------------------------------------------------------------------------------------
2020-05-11 15:48:38,766 EPOCH 1 done: loss 0.6263 - lr 0.1000
2020-05-11 15:49:03,715 DEV : loss 0.5499045848846436 - score 0.7266
2020-05-11 15:49:07,240 BAD EPOCHS (no improvement): 0
2020-05-11 15:49:11,766 ----------------------------------------------------------------------------------------------------
2020-05-11 15:49:12,288 epoch 2 - iter 0/5000 - loss 0.65329576 - samples/sec: 55976.48
2020-05-11 15:51:03,200 epoch 2 - iter 500/5000 - loss 0.59178817 - samples/sec: 150.89
2020-05-11 15:53:01,180 epoch 2 - iter 1000/5000 - loss 0.59182155 - samp

{'test_score': 0.7863,
 'dev_score_history': [0.7266,
  0.7409,
  0.7448,
  0.7527,
  0.7581,
  0.7591,
  0.7692,
  0.7607,
  0.7751,
  0.773,
  0.777,
  0.7788,
  0.7761,
  0.7742,
  0.7794],
 'train_loss_history': [0.6262768354952335,
  0.5823872073888778,
  0.5685568423330783,
  0.5591698117792606,
  0.5510672243237495,
  0.5471562961041927,
  0.5401257211267948,
  0.5378082399129868,
  0.5341873123556375,
  0.5305983853459358,
  0.5281305697679519,
  0.5256288422375918,
  0.5227320364534855,
  0.521134237754345,
  0.5181407861113548],
 'dev_loss_history': [tensor(0.5499),
  tensor(0.5265),
  tensor(0.5179),
  tensor(0.5046),
  tensor(0.4964),
  tensor(0.4937),
  tensor(0.4856),
  tensor(0.4975),
  tensor(0.4709),
  tensor(0.4763),
  tensor(0.4737),
  tensor(0.4667),
  tensor(0.4717),
  tensor(0.4707),
  tensor(0.4648)]}

# Testing out the model


In [24]:
from flair.data import Sentence

classifier = TextClassifier.load('model-saves/final-model.pt')

pos_sentence = Sentence(preprocess('I love Python!'))
neg_sentence = Sentence(preprocess('Python is the worst!'))

classifier.predict(pos_sentence)
classifier.predict(neg_sentence)

print(pos_sentence.labels, neg_sentence.labels)

2020-05-11 21:16:02,586 loading file model-saves/final-model.pt
[4 (0.9463609457015991)] [0 (0.8664907813072205)]
